# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [29]:
df = pd.read_pickle(r"Motor_Vehicle_Collisions_-_Crashes.pkl")
df


,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Show some exploratory data analysis here
***

In [45]:
df.describe()

,CRASH DATE_CRASH TIME,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,COLLISION_ID
count,2018245,1.788560e+06,1.788560e+06,2.018227e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06,2.018245e+06
mean,2017-05-14 17:39:03.899198976,4.062776e+01,-7.375228e+01,3.024249e-01,1.446306e-03,5.518507e-02,7.253827e-04,2.612468e-02,1.119785e-04,2.179889e-01,5.896212e-04,3.116455e+06
min,2012-07-01 00:05:00,0.000000e+00,-2.013600e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01
25%,2014-12-19 18:00:00,4.066792e+01,-7.397493e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.140681e+06
50%,2017-03-29 21:15:00,4.072102e+01,-7.392732e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.645346e+06
75%,2019-06-10 18:30:00,4.076956e+01,-7.386665e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.150156e+06
max,2023-08-15 23:59:00,4.334444e+01,0.000000e+00,4.300000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,4.300000e+01,5.000000e+00,4.655026e+06
std,NaN,1.980901e+00,3.727568e+00,6.937633e-01,4.007171e-02,2.412867e-01,2.741556e-02,1.614267e-01,1.062812e-02,6.549700e-01,2.648117e-02,1.503997e+06


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018245 entries, 0 to 2018244
Data columns (total 29 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   CRASH DATE_CRASH TIME          datetime64[ns]
 1   BOROUGH                        object        
 2   ZIP CODE                       object        
 3   LATITUDE                       float64       
 4   LONGITUDE                      float64       
 5   LOCATION                       object        
 6   ON STREET NAME                 object        
 7   CROSS STREET NAME              object        
 8   OFF STREET NAME                object        
 9   NUMBER OF PERSONS INJURED      float64       
 10  NUMBER OF PERSONS KILLED       float64       
 11  NUMBER OF PEDESTRIANS INJURED  int64         
 12  NUMBER OF PEDESTRIANS KILLED   int64         
 13  NUMBER OF CYCLIST INJURED      int64         
 14  NUMBER OF CYCLIST KILLED       int64         
 15  NUMBER OF MOTOR

In [47]:
df.shape

(2018245, 29)

In [48]:
df.columns

Index(['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5', 'VEHICLE_COMBINATION'],
      dtype='object')

In [49]:
# Mean
df.mean(numeric_only=True)

LATITUDE                         4.062776e+01
LONGITUDE                       -7.375228e+01
NUMBER OF PERSONS INJURED        3.024249e-01
NUMBER OF PERSONS KILLED         1.446306e-03
NUMBER OF PEDESTRIANS INJURED    5.518507e-02
NUMBER OF PEDESTRIANS KILLED     7.253827e-04
NUMBER OF CYCLIST INJURED        2.612468e-02
NUMBER OF CYCLIST KILLED         1.119785e-04
NUMBER OF MOTORIST INJURED       2.179889e-01
NUMBER OF MOTORIST KILLED        5.896212e-04
COLLISION_ID                     3.116455e+06
dtype: float64

In [50]:
# Median
df.median(numeric_only=True)

LATITUDE                         4.072102e+01
LONGITUDE                       -7.392732e+01
NUMBER OF PERSONS INJURED        0.000000e+00
NUMBER OF PERSONS KILLED         0.000000e+00
NUMBER OF PEDESTRIANS INJURED    0.000000e+00
NUMBER OF PEDESTRIANS KILLED     0.000000e+00
NUMBER OF CYCLIST INJURED        0.000000e+00
NUMBER OF CYCLIST KILLED         0.000000e+00
NUMBER OF MOTORIST INJURED       0.000000e+00
NUMBER OF MOTORIST KILLED        0.000000e+00
COLLISION_ID                     3.645346e+06
dtype: float64

## Borough with the second highest number of crashes reported since 2012

In [30]:
category_counts = df['BOROUGH'].value_counts()

In [31]:
print(category_counts)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64


## Borough with the minimum number of crashes adjusted for population

In [32]:
# Given data
borough_population = {
    'Bronx': 1446788,
    'Brooklyn': 2648452,
    'Manhattan': 1638281,
    'Queens': 2330295,
    'Staten Island': 487155
}

# Crashes data 
crashes_data = {
    'Bronx': 205345,   
    'Brooklyn': 441026,
    'Manhattan': 313266, 
    'Queens': 372457, 
    'Staten Island': 58297
}

# Calculate crash rates per 100,000 people
crash_rates = {borough: (crashes / borough_population[borough]) * 100000 for borough, crashes in crashes_data.items()}

# Find the borough with the lowest crash rate
max_borough = max(crash_rates, key=crash_rates.get)

# Print the result
print(f"The borough with the least crashes per 100,000 people is {max_borough} with a rate of {crash_rates[max_borough]:.2f}")

The borough with the least crashes per 100,000 people is Manhattan with a rate of 19121.63


## Analyzing the leading cause of crashes

In [33]:
filtered_df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

#alternative
#filtered_df= df.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1'])
#filtered_df = df[df['CONTRIBUTING FACTOR VEHICLE 1'].notna()]

filtered_df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
6,2021-12-14 17:05:00,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018238,2023-07-22 10:40:00,NaN,NaN,NaN,NaN,NaN,CLEARVIEW EXPRESSWAY,NORTHERN BOULEVARD,NaN,3.0,...,Unspecified,NaN,NaN,NaN,4647804,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
2018239,2023-06-16 00:00:00,NaN,NaN,40.854310,-73.930090,"(40.85431, -73.93009)",WEST 189 STREET,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4648255,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN


In [34]:
leading_cause = filtered_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().idxmax()

# Calculate the proportion of accidents attributable to the leading cause
total_accidents = len(filtered_df)
leading_cause_count = filtered_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().loc[leading_cause]
proportion = leading_cause_count / total_accidents

# Print the results
print(f"The leading cause of crashes is: {leading_cause}")
print(f"The proportion of accidents attributable to this cause is: {proportion:.2%}")

The leading cause of crashes is: Driver Inattention/Distraction
The proportion of accidents attributable to this cause is: 30.27%


## Top 5 causes of crashes 

In [35]:
causes_counts = filtered_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

top_5_causes = causes_counts.nlargest(5)

# Calculate the proportion of total crashes attributable to the top 5 causes
total_accidents = len(filtered_df)
top_5_proportion = top_5_causes.sum() / total_accidents

# Print the results
print("The top 5 causes of crashes (ignoring 'Unspecified') are:")
print(top_5_causes)
print(f"\nThe proportion of total crashes attributable to the top 5 causes is: {top_5_proportion:.2%}")

The top 5 causes of crashes (ignoring 'Unspecified') are:
CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    401262
Failure to Yield Right-of-Way     119166
Following Too Closely             107467
Backing Unsafely                   75042
Other Vehicular                    62688
Name: count, dtype: int64

The proportion of total crashes attributable to the top 5 causes is: 57.76%


### Total count of accidents that involved two or more fatalities

In [36]:
# Fill missing values for the number of persons killed with zero
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)

# Filter accidents with two or more fatalities
fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] >= 2]

# Calculate the total count of accidents with two or more fatalities
total_fatal_accidents = len(fatal_accidents)

# Print the result
print(f"The total count of accidents that involved two or more fatalities since 2012 is: {total_fatal_accidents}")


The total count of accidents that involved two or more fatalities since 2012 is: 88


### At least one person dead adjusted for accidents

In [37]:
fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] >= 1]

# Calculate the total number of accidents
total_accidents = len(df)

#Calculate the average for every 1000 accidents
total_fatal_accidents1 = len(fatal_accidents)

average_accidents = (total_fatal_accidents1 / total_accidents) * 1000

print(f"The average accidents per 1000 is: {average_accidents}")

The average accidents per 1000 is: 1.3893258747079764


### Accidents that do not have a Cross Street Name

In [38]:
# Calculate the number of accidents without a Cross Street Name
accidents_without_cross_street = len(df[df['CROSS STREET NAME'].isna()])

# Calculate the proportion of accidents without a Cross Street Name
proportion_without_cross_street = accidents_without_cross_street / total_accidents

# Print the result
print(f"The proportion of accidents without a Cross Street Name is: {proportion_without_cross_street:.2%}")

The proportion of accidents without a Cross Street Name is: 37.44%


### Combination of vehicles

In [39]:
# Combine the vehicle type codes from 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' into a new column
df['VEHICLE_COMBINATION'] = df['VEHICLE TYPE CODE 1'] + ' + ' + df['VEHICLE TYPE CODE 2']

# Find the combination of vehicles with the most number of accidents
most_common_combination = df['VEHICLE_COMBINATION'].value_counts().idxmax()

# Get the count of accidents for the most common combination
most_common_count = df['VEHICLE_COMBINATION'].value_counts().max()

# Print the result
print(f"The combination of vehicles with the most accidents is: {most_common_combination}")
print(f"The number of accidents for this combination is: {most_common_count}")

The combination of vehicles with the most accidents is: Sedan + Sedan
The number of accidents for this combination is: 197944


### Alcohol involvement in a fatality

In [40]:
# Filter the dataset to include only crashes with alcohol involvement as the contributing factor
alcohol_related_df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Calculate the total number of crashes with alcohol involvement
total_alcohol_related_crashes = len(alcohol_related_df)

# Calculate the number of crashes with alcohol involvement that resulted in a fatality
fatal_alcohol_related_crashes = len(alcohol_related_df[alcohol_related_df['NUMBER OF PERSONS KILLED'] > 0])

# Calculate the proportion of crashes with alcohol involvement that resulted in a fatality
proportion_fatalities_alcohol_related = fatal_alcohol_related_crashes / total_alcohol_related_crashes

# Print the result
print(f"The proportion of crashes with alcohol involvement resulting in a fatality is: {proportion_fatalities_alcohol_related:.2%}")

The proportion of crashes with alcohol involvement resulting in a fatality is: 0.47%


### Rush hour crashes

In [42]:
# Filter the dataset to include only crashes that occur between 4 PM and 7 PM
evening_rush_hour_df = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

# Calculate the total number of crashes
total_crashes = len(df)

# Calculate the number of crashes during the evening rush hour
crashes_in_evening_rush_hour = len(evening_rush_hour_df)

# Calculate the proportion of crashes during the evening rush hour
proportion_evening_rush_hour = crashes_in_evening_rush_hour / total_crashes

# Print the result
print(f"The proportion of crashes during the evening rush hour is: {proportion_evening_rush_hour:.2%}")

The proportion of crashes during the evening rush hour is: 20.51%


### Injuries but no fatalities crashes involving motorcycles

In [43]:
# Filter the dataset
moto_related_df = df[(df['VEHICLE TYPE CODE 1'] == 'MOTORCYCLE') | (df['VEHICLE TYPE CODE 2'] == 'MOTORCYCLE')]

# Calculate the number of crashes with injuries but no fatalities
fatal_moto_related_crashes = len(moto_related_df[(moto_related_df['NUMBER OF PERSONS KILLED'] == 0) & (moto_related_df['NUMBER OF PERSONS INJURED'] > 0)])

# Calculate the proportion
proportion_moto_crashes = fatal_moto_related_crashes / len(moto_related_df)

# Print the result
print(f"The proportion of crashes involving motorcycles is: {proportion_moto_crashes:.2%}")                  

The proportion of crashes involving motorcycles is: 50.05%


### Crashes involved bicycles as one of the vehicles

In [44]:
# Filter the dataset
bic_related_df = df[(df['VEHICLE TYPE CODE 1'] == 'BICYCLE') | (df['VEHICLE TYPE CODE 2'] == 'BICYCLE')]

#Calculate the number of crashes involved bicycles
bic_crashes =  len(bic_related_df)

#Print the result
print(f"The number of crashes involved bicycles is: {bic_crashes}")

The number of crashes involved bicycles is: 19108
